<h1>Data</h1>

In [ ]:
%matplotlib inline
from mysql import connector
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

mysql_connector = connector.connect(host="127.0.0.1", database="aq", 
                                    user="root", password="!QAZ2wsx#EDC")
data = pd.read_sql("""SELECT close, volume FROM future_trade
                      WHERE code='%s' AND type='%s' """ % ("RB", "5"), con=mysql_connector)
mysql_connector.close()
data["chg"] = data.close.diff()
data.iloc[0, 2] = 0

lag_rtn = 10
lag_vol = 1
y = data.chg[max(lag_rtn, lag_vol):]
y.index = range(len(y))
y = y.apply(lambda x: x > 0)
X = pd.DataFrame(columns=range(lag_rtn + lag_vol))
for idx, row in data.iterrows():
    if idx >= lag_rtn and idx >= lag_vol:
        vol = data.iloc[(idx - lag_vol):idx, 1]
        rtn = data.iloc[(idx - lag_rtn):idx, 2]
        xrow = pd.concat([rtn, vol], ignore_index=True)
        X = X.append(xrow, ignore_index=True)

print("Std:", data.chg.std())
print("Mean:", data.chg.mean())

# Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
k_fold = 20
stride = round(len(data.chg)/k_fold)
for i in range(0, k_fold):
    start = i*stride
    stop = i*stride+stride-1
    if (i == k_fold-1):
        stop = len(data.chg)
    X_test = X[start:stop]
    y_test = y[start:stop]
    X_train = pd.concat([X[0:start-1], X[stop:]])
    y_train = pd.concat([y[0:start-1], y[stop:]])    
    model = LogisticRegression()
    model.fit(X_train, y_train)
    print("k_fold=%s, %.2f%s" % (i, np.sum(model.predict(X_test) == y_test)*100/stride, "%"))

# Navie Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB
k_fold = 20
stride = round(len(data.chg)/k_fold)
for i in range(0, k_fold):
    start = i*stride
    stop = i*stride+stride-1
    if (i == k_fold-1):
        stop = len(data.chg)
    X_test = X[start:stop]
    y_test = y[start:stop]
    X_train = pd.concat([X[0:start-1], X[stop:]])
    y_train = pd.concat([y[0:start-1], y[stop:]])    
    model = GaussianNB()
    model.fit(X_train, y_train)
    print("k_fold=%s, %.2f%s" % (i, np.sum(model.predict(X_test) == y_test)*100/stride, "%"))